In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset
from nltk.tokenize import word_tokenize

c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset (SNLI - Natural Language Inference)
dataset = load_dataset("snli")

# Convert dataset to DataFrame
def dataset_to_dataframe(dataset):
    df = pd.DataFrame(dataset)
    return df

train_df = dataset_to_dataframe(dataset["train"])
test_df = dataset_to_dataframe(dataset["test"])

train_df.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0


In [4]:
# Extract text and labels
def preprocess_data(df, sample_size=5000):
    df_sample = df.dropna().sample(n=sample_size, random_state=42)
    texts = [p + " [SEP] " + h for p, h in zip(df_sample["premise"], df_sample["hypothesis"])]
    labels = df_sample["label"].tolist()
    return texts, labels

train_texts, train_labels = preprocess_data(train_df, sample_size=5000)
test_texts, test_labels = preprocess_data(test_df, sample_size=1000)

train_texts[0]

'An older gentleman in an orange jumpsuit and gray hat is on the curb with a black trash bag and a broom. [SEP] A man is cleaning up around his house.'

In [5]:
# 1. Transformer Embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Generating transformer embeddings...")
X_train_transformer = model.encode(train_texts, convert_to_numpy=True)
X_test_transformer = model.encode(test_texts, convert_to_numpy=True)

Generating transformer embeddings...


In [6]:
# 2. TF-IDF Features
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(train_texts).toarray()
X_test_tfidf = vectorizer.transform(test_texts).toarray()

In [7]:
# Train and Evaluate Models
def train_and_evaluate(X_train, X_test, train_labels, test_labels, method):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, train_labels)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(test_labels, y_pred)
    report = classification_report(test_labels, y_pred)
    print(f"{method} - Accuracy: {accuracy:.4f}")
    print(f"{method} - Classification Report:\n{report}\n")

print("Training and evaluating models...")
train_and_evaluate(X_train_transformer, X_test_transformer, train_labels, test_labels, "Transformer Embeddings")
train_and_evaluate(X_train_tfidf, X_test_tfidf, train_labels, test_labels, "TF-IDF")

Training and evaluating models...


c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Transformer Embeddings - Accuracy: 0.3890
Transformer Embeddings - Classification Report:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        17
           0       0.40      0.41      0.40       340
           1       0.36      0.30      0.33       333
           2       0.40      0.48      0.44       310

    accuracy                           0.39      1000
   macro avg       0.29      0.30      0.29      1000
weighted avg       0.38      0.39      0.38      1000


TF-IDF - Accuracy: 0.4600
TF-IDF - Classification Report:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        17
           0       0.49      0.55      0.52       340
           1       0.46      0.37      0.41       333
           2       0.43      0.48      0.45       310

    accuracy                           0.46      1000
   macro avg       0.34      0.35      0.35      1000
weighted avg       0.45      0.46   

c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shaak\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo